<a href="https://colab.research.google.com/github/jonasvdbran/Masterthesis-SCRPPP/blob/main/MILP_StochasticProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We again import the container_classes file. **Upload container_classes.py via the next block of code.** **bold text**

In [28]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
from container_classes import *

We again consider the same example bay to test our algorithms on.

In [ ]:
(B_max,S_max,L_max)=(1,4,4)
T_max=5

In [ ]:
module=Module([Bay([Stack([Container(1),Container(2)],4),
                    Stack([Container(3),Container(3),Container(4),Container(1)],4),Stack([Container(2)],4),Stack([Container(1),Container(4),Container(4),Container(5)],4)],4)],4)
print(module)

# MILP


In [ ]:
!apt-get install -y glpk-utils coinor-cbc


In [ ]:
from pyomo.environ import SolverFactory

solver = SolverFactory("cbc", executable="/usr/bin/cbc")  # Explicitly specify CBC path
if solver.available():
    print("CBC solver is available!")
else:
    print("CBC solver is NOT available.")


In [ ]:
from pyomo.environ import *

In [ ]:
def get_initial_positions(container_bay):
  #Convert the module into the LP format: initial_x[(t, (b, s, l))] = 1
  #Used to initialize LP problem
  initial_x = {}
  container_timeslots={}
  ID=0
  for b_idx, bay in enumerate(container_bay):
    for s_idx, stack in enumerate(bay):
      for l_idx, container in enumerate(stack):
          initial_x[(ID, (0,s_idx, l_idx))] = 1 # Stores container initial position
          container_timeslots[ID]=container.value # Maps container to its time slot
          ID+=1

  return initial_x, container_timeslots

In [ ]:
initial_x,container_timeslots=get_initial_positions(module)
print(initial_x)
print(container_timeslots)

In [ ]:
# Parameters
U_max = 7
alpha = 0.5

B= range(B_max) #Bays
S = range(S_max)  # Stacks
L = range(L_max)  # Levels
T = range(1, T_max + 1)  # Timestamps
U = range(U_max)  # States
C=range(len(initial_x))

# Define model
model = ConcreteModel()

# Decision Variables
model.x = Var(C,B,S,L,U, within=Binary)
model.w = Var(C,B,S,L,U[:-1], within=Binary) # Outgoing containers (moved from stack s)
model.z = Var(C,B,S,L,U[:-1], within=Binary) # Incoming containers (moved to stack s)

# Second stage variables
model.c = Var(B,S,L,within=NonNegativeReals)

# Objective function:
model.obj = Objective(expr=alpha*sum(model.w[c,b,s,l,tau] for c in C for b in B for s in S for l in L for tau in U[:-1])
    + sum(model.c[b,s,l] for b in B for s in S for l in L),sense=minimize)


# Constraint 0: Initial container positions
def initial_positions(model,c,b,s,l):
    return model.x[c,b,s,l,0] == initial_x.get((c, (b,s,l)), 0)

model.initial_positions_constraint = Constraint(C,B,S,L, rule=initial_positions)

# Contstraint 1: Number of incoming containers = number of outgoing containers
def balance_constraint(model,c,tau):
    return sum(model.w[c,b,s,l,tau] for b in B for s in S for l in L) == sum(model.z[c,b,s,l,tau] for b in B for s in S for l in L)

model.balance_constraint = Constraint(C,U[:-1],rule=balance_constraint)

# Constraint 2: At most 1 move of each kind per time unit
def at_most_one_w_move(model,tau):
    return sum(model.w[c,b,s,l,tau] for c in C for b in B for s in S for l in L) <= 1
def at_most_one_z_move(model,tau):
    return sum(model.z[c,b,s,l,tau] for c in C for b in B for s in S for l in L) <= 1

model.at_most_one_w_move_constraint = Constraint(U[:-1],rule=at_most_one_w_move)
model.at_most_one_z_move_constraint = Constraint(U[:-1],rule=at_most_one_z_move)

# Constraint: Moves are performed as early as possible.
#for tau in U[:-2]:
    #model.constraints.add(sum(model.w[t, s, l, tau+1] for s in S for l in L for t in T) <= sum(model.w[t, s, l, tau] for s in S for l in L for t in T))

# Constraint 3: At most one container per position + if there is a container, no container can come in.

def at_most_one_container_per_position(model,b,s,tau):
    if tau<U[-1]:
        return sum(model.x[c,b,s,0,tau] for c in C) + sum(model.z[c,b,s,0,tau] for c in C) <= 1
    else:
        return sum(model.x[c,b,s,0,tau] for c in C) <= 1

model.at_most_one_container_per_position_constraint = Constraint(B,S,U[1:],rule=at_most_one_container_per_position)

# Constraint 4: Controls topmost containers: if there is an outgoing move, the container isn't there in the next time unit.
# if there is an incoming move, container is there in the next time unit

def control_topmost_containers(model,c,b,s,l,tau):
  return model.x[c,b,s,l,tau] + model.z[c,b,s,l,tau] == model.x[c,b,s,l,tau+1] + model.w[c,b,s,l,tau]

model.control_topmost_containers_constraint = Constraint(C,B,S,L,U[:-1],rule=control_topmost_containers)

# Constraint 5: Can't move if there is a container above + no floating containers

def no_floating_containers(model,b,s,l,tau):
  return sum(model.x[c,b,s,l+1,tau] for c in C) + sum(model.w[c,b,s,l,tau] for c in C) + sum(model.z[c,b,s,l+1,tau] for c in C) <= sum(model.x[c,b,s,l,tau] for c in C)

model.no_floating_containers_constraint = Constraint(B,S,L[:-1],U[:-1],rule=no_floating_containers)

# Second stage costs -> shifting costs:

# Constraint 6: Counts the number of misplaced containers
def blocking_containers(model, b, s, l, l_prime,t):
    if l_prime >= l:
        return Constraint.Skip
    return model.c[b,s,l] >= 1.2*sum(model.x[c,b, s, l,U_max-1] - model.x[c,b, s, l_prime,U_max-1]
                for c in C if container_timeslots[c]>=t)

model.blocking_containers_constraint = Constraint(B, S, L[1:], L,T, rule=blocking_containers)

# Constraint 7: Adds a cost of 0.5 if there is a container underneath with same time slot
def same_time_slot(model, b, s, l, l_prime,t):
    if l_prime >= l:
        return Constraint.Skip
    else:
        return model.c[b,s,l] >= 0.5*(sum(model.x[c,b, s, l,U_max-1] for c in C if container_timeslots[c]==t)-
                               sum(model.x[c,b, s, l_prime,U_max-1] for c in C if container_timeslots[c]<t)-
                               sum(model.x[c,b, s, l_prime,U_max-1] for c in C if container_timeslots[c]>t))
model.cost_shift_constraint = Constraint(B, S, L[1:], L,T, rule=same_time_slot)

In [ ]:
# Solve the model
solver = SolverFactory("cbc", executable="/usr/bin/cbc")
solver.solve(model)

In [ ]:
print(module)

We apply the moves given by the solution of the MILP via the variables w and z.



In [ ]:
def apply_moves(container_bay):
  new_bay=[]
  for timestep in U[:-1]:
    from_stack = next(((b,s) for (c,b, s, l, tau), var in model.w.items() if tau == timestep and var.value == 1),None)
    destination_stack=next(((b,s) for (c, b,s, l, tau), var in model.z.items() if tau == timestep and var.value == 1), None)
    print(f"tau: {timestep} -> from {from_stack} to {destination_stack}")

    if from_stack is not None and destination_stack is not None:
      if not new_bay:
        new_bay=container_bay.move(from_stack,destination_stack,in_place=False)
      else:
        new_bay=new_bay.move(from_stack,destination_stack,in_place=False)

    if from_stack is None and destination_stack is not None:
      raise ValueError("No container to move")

    if from_stack is not None and destination_stack is None:
      raise ValueError("No destination stack")

    if from_stack is None and destination_stack is None:
      continue
  return new_bay if new_bay else container_bay

In [ ]:
new_module=apply_moves(module)
print(new_module)

We can check if the final positions resulting from the variables w and z are indeed the same as the x indices at the final state.

In [ ]:
x_one_indices = [idx for idx in model.x if value(model.x[idx]) ==1 and idx[4]==U_max-1]

print("x variables at final state equal to 1:")
for idx in x_one_indices:
    print(f"x{idx}")

# Two-stage stochastic program

In [ ]:
# Define scenarios (Ω) with probability p_omega
max_delay=3
max_scenarios=50
possible_delays=[i for i in range(max_delay+1)]
prob_delays=[0.1/delay for delay in possible_delays if not delay==0]
prob_no_delay=1-sum(prob_delays)

Omega = range(max_scenarios)
# Every scenario corresponds to a row. Number i in the row represents the delay given to container 1.
delays = np.random.choice(possible_delays, size=(len(Omega), len(C)), p=[prob_no_delay]+prob_delays)

In [ ]:
delays

In [ ]:
def get_scenario_time_slots(container_timeslots,delays):
  scenario_time_slots={}
  for scenario in Omega:
    for c in C:
      scenario_time_slots[c,scenario]=container_timeslots[c]+delays[scenario,c]
  return scenario_time_slots

In [ ]:
scenario_time_slots=get_scenario_time_slots(container_timeslots,delays)
print(scenario_time_slots)

In [ ]:
model = ConcreteModel()

###------------- Parameters -------------###
U_max = 6
alpha = 0.6
MC=1.3

B= range(B_max) #Bays
S = range(S_max)  # Stacks
L = range(L_max)  # Levels
T = range(1, T_max + max_delay+1)  # Timestamps
U = range(U_max)  # States
C= range(len(initial_x))



###------------- Variables -------------###
# First stage decision Variables
model.x = Var(C,B,S,L,U, within=Binary)
model.w = Var(C,B,S,L,U[:-1], within=Binary) # Outgoing containers (moved from stack s)
model.z = Var(C,B,S,L,U[:-1], within=Binary) # Incoming containers (moved to stack s)

# Second stage variables
model.c = Var(B,S,L,Omega,within=NonNegativeReals)


###------------- Objective function -------------###

N=len(Omega) # number of scenarios

# First stage
model.firstStageCost = Expression(
    expr=alpha * sum(model.z[c,b,s,l,tau] for c in C for b in B for s in S for l in L for tau in U[:-1]))
# Second stage
model.secondStageCost = Expression(
    expr=sum(model.c[b,s, l, scenario] for b in B for s in S for l in L for scenario in Omega) / N)

# Total objective function: first+second stage
model.obj = Objective(expr=model.firstStageCost + model.secondStageCost,sense=minimize)


###------------- Constraints -------------###
# Constraint 0: Initial container positions
def initial_positions(model,c,b,s,l):
    return model.x[c,b,s,l,0] == initial_x.get((c, (b,s,l)), 0)

model.initial_positions_constraint = Constraint(C,B,S,L, rule=initial_positions)

# Contstraint 1: Number of incoming containers = number of outgoing containers
def balance_constraint(model,c,tau):
    return sum(model.w[c,b,s,l,tau] for b in B for s in S for l in L) == sum(model.z[c,b,s,l,tau] for b in B for s in S for l in L)

model.balance_constraint = Constraint(C,U[:-1],rule=balance_constraint)

# Constraint 2: At most 1 move of each kind per time unit
def at_most_one_w_move(model,tau):
    return sum(model.w[c,b,s,l,tau] for c in C for b in B for s in S for l in L) <= 1
def at_most_one_z_move(model,tau):
    return sum(model.z[c,b,s,l,tau] for c in C for b in B for s in S for l in L) <= 1

model.at_most_one_w_move_constraint = Constraint(U[:-1],rule=at_most_one_w_move)
model.at_most_one_z_move_constraint = Constraint(U[:-1],rule=at_most_one_z_move)

# Constraint: Moves are performed as early as possible.
#for tau in U[:-2]:
    #model.constraints.add(sum(model.w[t, s, l, tau+1] for s in S for l in L for t in T) <= sum(model.w[t, s, l, tau] for s in S for l in L for t in T))

# Constraint 3: At most one container per position + if there is a container, no container can come in.

def at_most_one_container_per_position(model,b,s,tau):
    if tau<U[-1]:
        return sum(model.x[c,b,s,0,tau] for c in C) + sum(model.z[c,b,s,0,tau] for c in C) <= 1
    else:
        return sum(model.x[c,b,s,0,tau] for c in C) <= 1

model.at_most_one_container_per_position_constraint = Constraint(B,S,U[1:],rule=at_most_one_container_per_position)

# Constraint 4: Controls topmost containers: if there is an outgoing move, the container isn't there in the next time unit.
# if there is an incoming move, container is there in the next time unit

def control_topmost_containers(model,c,b,s,l,tau):
  return model.x[c,b,s,l,tau] + model.z[c,b,s,l,tau] == model.x[c,b,s,l,tau+1] + model.w[c,b,s,l,tau]

model.control_topmost_containers_constraint = Constraint(C,B,S,L,U[:-1],rule=control_topmost_containers)

# Constraint 5: Can't move if there is a container above + no floating containers

def no_floating_containers(model,b,s,l,tau):
  return sum(model.x[c,b,s,l+1,tau] for c in C) + sum(model.w[c,b,s,l,tau] for c in C) + sum(model.z[c,b,s,l+1,tau] for c in C) <= sum(model.x[c,b,s,l,tau] for c in C)

model.no_floating_containers_constraint = Constraint(B,S,L[:-1],U[:-1],rule=no_floating_containers)

###------------- Second stage costs: shifting costs -------------###

# Constraint 6: Counts the number of misplaced containers
def blocking_containers(model, b, s, l, l_prime,t,scenario):
    if l_prime >= l:
        return Constraint.Skip
    return model.c[b,s,l,scenario] >= MC*(sum(model.x[c,b, s, l,U_max-1] for c in C if scenario_time_slots[c,scenario]>=t) - sum(model.x[c,b, s, l_prime,U_max-1]for c in C if scenario_time_slots[c,scenario]>=t))


model.blocking_containers_constraint = Constraint(B, S, L[1:],L,T,Omega, rule=blocking_containers)

# Constraint 7: Adds a cost of 0.5 if there is a container underneath with same time slot
def same_time_slot(model, b, s, l, l_prime,t,scenario):
    if l_prime >= l:
        return Constraint.Skip
    else:
        return model.c[b,s,l,scenario] >= 0.5*(sum(model.x[c,b, s, l,U_max-1] for c in C if scenario_time_slots[c,scenario]==t)-
                               sum(model.x[c,b, s, l_prime,U_max-1] for c in C if scenario_time_slots[c,scenario]<t)-
                               sum(model.x[c,b, s, l_prime,U_max-1] for c in C if scenario_time_slots[c,scenario]>t))
model.cost_shift_constraint = Constraint(B, S, L[1:],L,T,Omega, rule=same_time_slot)

In [ ]:
# Solve the model
solver = SolverFactory("cbc",executable="/usr/bin/cbc")
solver.solve(model)

In [ ]:
from pyomo.environ import value

print(f"First stage objective value: {value(model.firstStageCost)}")
print(f'Second stage objective value: {value(model.secondStageCost)}')

**Apply Moves**

In [ ]:
new_module=apply_moves(module)
print(new_module)

In [ ]:
x_one_indices = [idx for idx in model.x if value(model.x[idx]) ==1 and idx[4]==U_max-1]

print("x variables at final state equal to 1:")
for idx in x_one_indices:
    print(f"x{idx}")

We can also take a look at the modules obtained in the different scenarios and their respective shifting costs.

In [ ]:
def estimate_shiftings(module):
  return sum(CalculateShiftingCosts(module)[0].values())

In [ ]:
def get_scenarios():
  for scenario in Omega:
    scenario_module=[]
    for b in B:
      bay=[]
      for s in S:
        stack=[]
        for l in L:
          c_list=[c for c in C if model.x[c,b,s,l,U_max-1].value==1]
          if c_list:
            c=c_list[0]
            stack.append(Container(scenario_time_slots[c,scenario]))
          else:
            break
        bay.append(Stack(stack,L_max))
      scenario_module.append(Bay(bay,L_max))
    module=Module(scenario_module,L_max)
    print(f"Scenario {scenario}:")
    print(module)
    print(f'Shifting Costs: {estimate_shiftings(module)}')

In [ ]:
get_scenarios()